In [1]:
import json
import pandas as pd
import re
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

import os
from IPython.display import display, Image

In [7]:
year =  [str(i) for i in range(1980, 2016) ]

df = pd.DataFrame()

for y in tqdm(year):
    file_path = 'movie_data_per_year/'+y+'.txt'  # Replace with the path to your text file


        
    input_file = file_path  # Replace with the path to your input file
    output_file = file_path  # Replace with the path to your output file

        
    
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

        
    
    jump = 0
    
    while not("}" in lines[jump]):
        jump += 1
        
    jump += 1
        
        
    for i in range(0, len(lines), jump):
        test_text = lines[i: i+jump]
        fin_text  = ''.join(test_text)
        

        cleaned_text = re.sub(r'ObjectId\("([^"]+)"\)', r'"\1"', fin_text)

        # Parse the text as a JSON object
        json_obj = json.loads(cleaned_text)
        # Convert the JSON object to a pandas DataFrame
        json_obj["imdbID"] += ".jpg"
        df = df.append(json_obj, ignore_index = True)

df = df.drop("Poster", axis = 1)
df = df.rename(columns={'imdbID': 'Poster'})

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:30<00:00,  1.20it/s]


In [8]:
df.to_csv("complete_data_movie.csv", encoding='utf-8', index=False)

In [128]:
df_movie = pd.read_csv("complete_data_movie.csv")

In [129]:
#only keep rows with movie type (remove series and games)
df_movie = df_movie[df_movie["Type"] == "movie"]


# changes actors and directors to lists and remove useless columns (id, Rated, imdbVotes, Response, type, Metascore)
df_movie['Actors'] = df_movie['Actors'].map(lambda x: x.split(', ') if isinstance(x, str) else np.nan)
df_movie['Director'] = df_movie['Director'].fillna('').apply(lambda x: [director.strip() for director in x.split(',')])
df_movie = df_movie[['Title','Genre', 'Actors','Director','Writer','Language','Box_office','Country','Released','Runtime','imdbRating','Awards', 'Poster']]

#drop cols with no actors / genre / poster
df_movie = df_movie.dropna(subset=['Actors','Genre','Poster', 'Released', 'imdbRating', 'Box_office'])

df_movie[0:2]

,Title,Genre,Actors,Director,Writer,Language,Box_office,Country,Released,Runtime,imdbRating,Awards,Poster
0,Stir Crazy,"Comedy, Crime","[Gene Wilder, Richard Pryor, Georg Stanford Br...",[Sidney Poitier],Bruce Jay Friedman,English,"101,500,000",USA,12 Dec 1980,111 min,6.8,1 nomination.,tt0081562.jpg
1,Airplane!,Comedy,"[Kareem Abdul-Jabbar, Lloyd Bridges, Peter Gra...","[Jim Abrahams, David Zucker, Jerry Zucker]","Jim Abrahams (written for the screen by), Davi...",English,"83,400,000",USA,02 Jul 1980,88 min,7.8,Nominated for 1 Golden Globe. Another 2 wins &...,tt0080339.jpg


In [130]:
actor_df = pd.read_csv("actorfilms.csv") #to update the actors list 


In [131]:
def get_actors_by_film(df, film, year):
    return list(df[(df["Film"] == film) & (df["Year"] == year)]["Actor"].values)


same_title = []

for title in tqdm(df_movie["Title"].unique()):
    bulk_year = int(df_movie[df_movie['Title'] == title]["Released"].values[0][-4:])
    
    if title in actor_df[actor_df["Film"] == title]["Film"].unique():
        
        for upt_year in actor_df[actor_df["Film"] == title]["Year"].unique():
        
            if abs(bulk_year - upt_year) < 2:
                title_index = df_movie[(df_movie["Title"] == title) & (df_movie["Released"].str.contains(str(bulk_year)))].index[0]
                df_movie.at[title_index, 'Actors'] = get_actors_by_film(actor_df, title, bulk_year)

            else:
                    
                new_row = {"Title": title, 
                           "Year_bulk": [bulk_year],
                           "Year_upt": [upt_year]
                          }
                same_title.append(new_row)


100%|██████████████████████████████████████████████████████████████████████████████| 6012/6012 [05:14<00:00, 19.09it/s]


In [142]:
df_movie.to_csv("complete_data_movie_with_correct_actor.csv", encoding='utf-8', index=False)


In [9]:
t = {
    1: 
        {
            2: [10], 
            3: [11], 
            4: [12], 
            'Own_movies': [10,11,12]
        }
}

In [10]:
actors_dict = {"Brad Pitt": 1, "Matt Damon": 2, "Leo": 3, "Jean": 4}
movies_dict = {"f1": 10, "f2": 11, "f3": 12}

In [35]:
def get_actor(name, info_dict):
    actor_id = actors_dict.get(name)
    played_with = list(info_dict.get(actor_id).keys())[:-1]
    shared_movies = info_dict.get(actor_id).get("Own_movies")

    return info_dict[actor_id], played_with,shared_movies
    

In [36]:
get_actor("Brad Pitt", t)

({2: [10], 3: [11], 4: [12], 'Own_movies': [10, 11, 12]},
 [2, 3, 4],
 [10, 11, 12])

In [ ]:
 import json
with open('result.json', 'w') as fp:
    json.dump(sample, fp)